In [190]:
import torch
import json
import cv2
from datetime import date
import numpy as np
import os
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from skimage import measure
from ibug.face_detection import RetinaFacePredictor
from ibug.face_parsing import FaceParser as RTNetPredictor
%matplotlib inline

In [191]:
threshold = 0.8 # default = 0.8
weights = None # r"C:\mahmoud_dev\machine learning\segmentation\face_parsing\ibug\face_parsing\rtnet\weights\rtnet101-fcn-14.torch" # default = None
num_classes = 11 # default = 11
max_num_faces = 50 # default = 50

parser_encoder = 'rtnet50'
parser_decoder = 'fcn'
rotate_image = False
today = date.today()


if torch.cuda.is_available():
    device = 'cuda:0'
face_detector = RetinaFacePredictor(threshold=threshold, device=device, model=(RetinaFacePredictor.get_model('mobilenet0.25')))
face_parser = RTNetPredictor(device=device, ckpt=weights, encoder=parser_encoder, decoder=parser_decoder, num_classes=num_classes)

def get_image_pred(img, face_detector, face_parser):
    if rotate_image:
        img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)

    faces = face_detector(img, rgb=False)
    masks = face_parser.predict_img(img, faces, rgb=False)
    
    return faces, masks


Hybrid stages [True, True, True]


In [192]:
categories_list = ['background', 'skin', 'left_eyebrow', 'right_eyebrow', 'left_eye','right_eye',
                   'nose', 'upper_lip', 'inner_mouth', 'lower_lip', 'hair'] # , 'left_ear', 'right_ear',  'glasses']

categories = [{"name":category, "id":i} for i, category in enumerate(categories_list)]


# Initialize COCO format dictionary
coco_dict = {
    'info': {"contributor":"mahmoud tabikh","date_created":date.today().strftime('%d-%m-%Y'),"description":"", "version":""},
    'licenses': [],
    'categories': categories,
    'images': [],
    'annotations': []
}

In [193]:
# Set paths and filenames
image_dir = r'D:\_Xchng\Mahmoud\segmenation\dataset\data\raw_images'
json_filepath = r'D:\_Xchng\Mahmoud\segmenation\dataset\data\instances_default.json'
txt_filepath = r'D:\_Xchng\Mahmoud\segmenation\dataset\data\yolo_annotations\obj_train_data'

In [194]:
# different segmentation methods

def get_segmenation_xy(mask, class_id):
    # Create segmentation and bbox arrays
    mask_bool = mask == class_id
    area = np.sum(mask_bool)

    ys, xs = np.where(mask_bool)
    segmentation = np.asarray(list(zip(xs, ys))).flatten().tolist()
    return area, segmentation

def working_get_segmentation_countours(mask, class_id):
    # Create segmentation and bbox arrays
    mask_bool = mask == class_id
    area = np.sum(mask_bool)

    retrieval_method = cv2.RETR_EXTERNAL # options: cv2.RETR_EXTERNAL, cv2.RETR_TREE
    contour_approximation = cv2.CHAIN_APPROX_SIMPLE # options: cv2.CHAIN_APPROX_SIMPLE, cv2.CHAIN_APPROX_NONE
    contours, hierarchy = cv2.findContours(mask_bool.astype(np.uint8), retrieval_method, contour_approximation)
    segmentation = []
    for contour in contours:
        segmentation.extend(contour.flatten().tolist())
    return area, segmentation

def get_segmentation_countours(mask, class_id):
    # Create segmentation and bbox arrays
    mask_bool = mask == class_id
    area = np.sum(mask_bool)

    retrieval_method = cv2.RETR_EXTERNAL # options: cv2.RETR_EXTERNAL, cv2.RETR_TREE
    contour_approximation = cv2.CHAIN_APPROX_SIMPLE # options: cv2.CHAIN_APPROX_SIMPLE, cv2.CHAIN_APPROX_NONE
    contours, hierarchy = cv2.findContours(mask_bool.astype(np.uint8), retrieval_method, contour_approximation)
    segmentation = []
    for contour in contours:
        segmentation.extend(contour.flatten().tolist())
    return area, segmentation

In [195]:
def save_segmentation_coco(image_dir):
    # Loop through images in directory
    for image_id, filename in enumerate(os.listdir(image_dir)):
        if filename.endswith(tuple([".png", ".jpg"])):
            image_path = os.path.join(image_dir, filename)
            image = cv2.imread(image_path)
            image_dict = { 
                'id': image_id+1,
                'width': image.shape[1],
                'height': image.shape[0],
                'file_name': filename}
            coco_dict['images'].append(image_dict)
            faces, masks = get_image_pred(image, face_detector, face_parser)
            mask_arr, face = masks[0], faces[0] # assumes 1 face per image, loop for more faces.
            annotations_list = get_annotations_list(mask_arr, face, image_id+1, coco_dict["annotations"])
            for dict_ in annotations_list:
                coco_dict['annotations'].append(dict_)
    save_json(coco_dict)


def get_annotations_list(mask, face, image_id, coco_annotation_dict):
    # Create annotation dictionary for each unique mask value
    annotation_list = [] # i am making a list of lists instead of list of dicts
    for class_id in np.unique(mask):
        if class_id != 0:
            annotation_id = len(coco_annotation_dict) + 1
            annotation_dict = {
                'id': annotation_id,
                'image_id': image_id,
                'category_id': class_id,
                'segmentation': [],
                'bbox': face[:4].astype(int),
                'iscrowd': 0}
            area, segmentation = get_segmentation_countours(mask, class_id)
            annotation_dict['segmentation'].append(segmentation)
            annotation_dict['area'] = area
            annotation_list.append(annotation_dict)
    # Add annotation to COCO dictionary
    return annotation_list

def save_json(coco_dict):
    class NpEncoder(json.JSONEncoder):
        def default(self, obj):
            if isinstance(obj, np.integer):
                return int(obj)
            if isinstance(obj, np.floating):
                return float(obj)
            if isinstance(obj, np.ndarray):
                return obj.tolist()
            return super(NpEncoder, self).default(obj)
    Path(json_filepath).write_text(json.dumps(coco_dict, cls=NpEncoder, indent=3))
    print(f"json file written to {json_filepath}")

In [196]:
save_segmentation_coco(image_dir)

json file written to D:\_Xchng\Mahmoud\segmenation\dataset\data\instances_default.json
